In [5]:
# Import libraries
import numpy as np
import sqlite3
import random
import ast
import pandas as pd

In [8]:
dataframes = [] #Here will import that raw data from the article

min_num_vertices = 4
max_num_vertices = 4 #Set to 3 when generating everything from 3-6

#Import Polygon Data. # It verifies we have the same number of vertices across the database.
with sqlite3.connect("data/dim_2_plucker.db") as db: #...ensure this is the correct path to the datafile
    for i in range(min_num_vertices, max_num_vertices+1): #Iterate through n=3 to n=6, where n is the number of vertices
      #c = db.cursor()
      query = "SELECT * FROM dim_2_plucker WHERE num_vertices = {}".format(i)
      dataframes.append(pd.read_sql_query(query, db))   #...read database into a pandas dataframe
      headings = dataframes[(i - min_num_vertices)].columns.values     #...save the headings of each column in the table
      data = dataframes[(i - min_num_vertices)].values

OperationalError: unable to open database file

In [ ]:
dataframes[0]

,id,vertices,plucker,plucker_len,num_vertices,num_points,num_interior_points,volume,dual_volume,gorenstein_idx,codimension,embedding
0,469,"[[1,0],[-1,0],[0,1],[1,-1]]","[-1,-1,1,0,1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
1,470,"[[1,0],[-1,0],[0,1],[1,-1]]","[0,-1,-1,-1,-1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
2,471,"[[1,0],[-1,0],[0,1],[1,-1]]","[1,0,-1,1,1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
3,1009,"[[1,0],[-1,0],[2,5],[-5,-8]]","[-8,-8,-9,0,5,5]",6,4,18,10,26,13/10,20,10,"[1,2,2,2,3,3,3,3,4,4,4,5,5]"
4,1010,"[[1,0],[-1,0],[2,5],[-5,-8]]","[-8,-5,0,9,8,5]",6,4,18,10,26,13/10,20,10,"[1,2,2,2,3,3,3,3,4,4,4,5,5]"
...,...,...,...,...,...,...,...,...,...,...,...,...
21118,80979,"[[1,0],[116,125],[-12,-13],[-1,-1]]","[-1,13,125,1,9,8]",6,4,89,61,148,16/5,25,6,"[1,1,1,1,1,9,13,17,25]"
21119,80980,"[[1,0],[116,125],[-12,-13],[-1,-1]]","[1,-1,-9,-13,-125,8]",6,4,89,61,148,16/5,25,6,"[1,1,1,1,1,9,13,17,25]"
21120,80981,"[[1,0],[139,180],[3,4],[-27,-35]]","[-16,-180,5,4,3,35]",6,4,130,106,234,1,30,8,"[1,1,2,3,5,9,9,13,16,23,30]"
21121,80982,"[[1,0],[139,180],[3,4],[-27,-35]]","[5,16,180,3,35,4]",6,4,130,106,234,1,30,8,"[1,1,2,3,5,9,9,13,16,23,30]"


In [ ]:
max_index = max_num_vertices - min_num_vertices + 1
print(max_index)

1


In [ ]:
for j in range(0, max_index):
  #selects the features that we are looking into, and eliminates duplicates
  new_df = dataframes[j][['vertices', 'num_vertices', 'volume']]
  new_df['vertices_1'] = new_df['vertices'].apply(lambda x: ast.literal_eval(x))
  #new_df = new_df.drop_duplicates(subset='vertices_1')
  new_df['coordinates_1'] =   new_df['vertices_1'].apply(lambda x: [item for sublist in x for item in sublist])

  V_dfs = []

  for k in range(2,3+1): #number of random orientations of the vertices
    vert = 'vertices_{}'.format(k)
    coord = 'coordinates_{}'.format(k)
    new_df[vert]  =   new_df['vertices_1'].apply(lambda x: random.sample(x, len(x)))
    new_df[coord] =   new_df[vert].apply(lambda x: [item for sublist in x for item in sublist])

    V_df = new_df[[vert, coord, "volume"]]
    V_df = V_df.rename(columns={vert: 'vertices', coord: 'coordinates'})

    V_dfs.append(V_df)

  Final_df = pd.concat(V_dfs, axis=0)

<ipython-input-5-7296f99e87c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['vertices_1'] = new_df['vertices'].apply(lambda x: ast.literal_eval(x))
<ipython-input-5-7296f99e87c3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['coordinates_1'] =   new_df['vertices_1'].apply(lambda x: [item for sublist in x for item in sublist])
<ipython-input-5-7296f99e87c3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [ ]:
Final_df.tail(2)

,vertices,coordinates,volume
21121,"[[-27, -35], [139, 180], [1, 0], [3, 4]]","[-27, -35, 139, 180, 1, 0, 3, 4]",234
21122,"[[1, 0], [-27, -35], [139, 180], [3, 4]]","[1, 0, -27, -35, 139, 180, 3, 4]",234


In [ ]:
# Next, we creates a gigantic database with coordinates and vertices for all the transformed databases B_k*D
#It goes until

k_max = 100

for i in range(0, k_max+1):
    # Define the name of the new columns
    new_column_name = f'verticesB_{i}'
    new_coordinate_name = f'coordinatesB_{i}'

    # Define the matrix B
    B = np.array([[1, i], [0, 1]])

    # Define a lambda function to multiply each element of the nested list by B
    multiply_by_B = lambda lst: [np.dot(B, v) for v in lst]

    # Create a new column
    Final_df[new_column_name]     =   Final_df['vertices'].apply(multiply_by_B)
    Final_df[new_coordinate_name] =   Final_df[new_column_name].apply(lambda x: [item for sublist in x for item in sublist])

Final_df.head()

SyntaxError: ignored

In [ ]:
database_name = '/content/{0}Vertices_k{1}.db'.format(1 + 3, k_max)
conn = sqlite3.connect(database_name)
data_toSQL = Final_df.applymap(str)
data_toSQL.to_sql('Data', conn, if_exists='replace')
conn.close()

#Checking the database has the correct Polygon Data.
conn = sqlite3.connect(database_name)
# Read a table from the database into a Pandas DataFrame
check_df = pd.read_sql('SELECT * FROM Data', conn)
# Close the database connection
conn.close()
check_df.head()